# **Spatially Resolved Transcriptomics Datasets Measured Utilizing Different Platforms.**

In this tutorial, we demonstrate how to apply `BatchEval` to assess integrated data that is measured bt different platforms. As an example, we used three mouse olfactory bulb datas. One tissue section was profiled by `10x Genomics Visium`, while the others were measured by `Stereo-seq`.           

github: [https://github.com/STOmics/BatchEval.git](https://github.com/STOmics/BatchEval.git)

### Import packages

In [1]:
import os
import sys

In [2]:
sys.path.append(os.getcwd())

In [3]:
import scanpy as sc
from anndata import AnnData
from warnings import filterwarnings
from BatchEval import batch_eval
filterwarnings("ignore")

### Load datasets

In [4]:
data = [sc.read_h5ad(os.path.join("./demo_data", t)) for t in sorted(os.listdir("./demo_data")) if t.endswith("h5ad")]

### Optional: External data integration methods

In [5]:
import scanorama

In [6]:
merge_data = AnnData.concatenate(*data)

In [7]:
scanorama_correct = scanorama.correct_scanpy(
    [merge_data[merge_data.obs["batch"] == c] for c in merge_data.obs["batch"].cat.categories], 
    return_dimred=True)

Found 22891 genes among all datasets
[[0.         0.50985222 0.        ]
 [0.         0.         0.02111486]
 [0.         0.         0.        ]]
Processing datasets (0, 1)


In [8]:
scanorama_correct = AnnData.concatenate(*scanorama_correct)

### Running `BatchEval` Pipeline

In [ ]:
batch_eval(*data,
           norm_log=True,
           is_scale=False,
           n_pcs=50,
           n_neighbors=15,
           batch_key="batch",
           position_key="X_umap",
           condition=None,
           count_key="total_counts",
           celltype_key="celltype",
           external_list=[{
               "correct_data": scanorama_correct,
               "re_pca": False,
               "re_neigh": True,
               "use_rep": "X_scanorama",
               "re_umap": True,
               "batch_key": "batch"},],
           report_path="./output/mouse_ob")

2023-09-18 16:00:18 BatchEval Pipeline Starting


2023-09-18 16:00:32.389107: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 16:00:37.907511: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-09-18 16:05:57 The 'Raw Report.html' has been saved to ./output/mouse_ob


### Ouput          
A series of HTML pages were output for the data integration evaluations, which we are able to find in the`report_path`.    
                            
- `BatchEval Report.html`: the summaries report of `BatchEval Pipeline`.           
- `Raw Report.html`: the raw data integration report.
- `Harmony Report.html`: the data integration report of using `BatchEval Pipeline` built-in `Harmony` mehtod, which is single cell base methods.
- `BBKNN Report.html`: the data integration report of using `BatchEval Pipeline` built-in `BBKNN` mehtod, which is single cell base methods.                 
- `spatiAlign Report.html`: the data integration report of using `BatchEval Pipeline` built-in `spatiAlign` mehtod, which is spatially resolved transcriptomics method.
- `External Report.html`: if the `externnal_list` is not `None`, and will generate external report.